In [8]:
import pandas as pd
import json
import os, glob
from tqdm import tqdm
dataset = []


# Apply labels to self-supervised set for detectron2 #

In [1]:
import pandas as pd
df = pd.read_parquet('/data/yelena/oct-semisup/data/dfselfsup.parquet',engine='fastparquet')
# dfspecial = df[df.age==90.0100].groupby(['ptid','eye'])['instance'].count().sort_values(ascending=False)
# dfspecial[dfspecial>1]
df = df[df.age<90.0100] #max age == lots of cross-talk between labels, exclude
df = df.drop(columns=['slicei','fpath'])
df = df.loc[df.astype(str).drop_duplicates().index]
#merge on dx labels
dflabels = pd.read_parquet('/data/yelena/oct-semisup/data/dflabels_v2_procedure.parquet',engine='fastparquet')
dflabels = dflabels.fillna('NA')

In [2]:
#eyes after injection to exclude
inj_eye = dflabels[dflabels['PROC_CODE']== '67028'][['ptid','eye','age']].drop_duplicates()
inj_eye = inj_eye.groupby(['ptid','eye']).min()

dfexclude = dflabels.merge(inj_eye,on=['ptid','eye'],suffixes=['','_inj']).query('age>=age_inj').drop('age_inj',axis=1)[['ptid','eye','age']].drop_duplicates()
dflabels = dflabels[['ptid','eye','age','diagnosis']].groupby(['ptid','eye','age']).aggregate(lambda dx: dx.unique().tolist())
df = df.merge(dflabels,how='left', on=['ptid','eye','age']) #this assumes dx is a list
#exclude eyes after they've received injection
df = df.merge(dfexclude,how='left',on=['ptid','eye','age'],indicator=True).query('_merge=="left_only"').drop('_merge',axis=1)
#select AMD dx
df = df[df.diagnosis.apply(lambda x: 'AMD' in x)]
df.astype(str).nunique()

ptid         1428
eye             2
age          4591
instance     9339
diagnosis     339
dtype: int64

In [3]:
#sample 1 volume per ptid
dfsample = df.sample(frac=1,random_state=4).drop_duplicates(subset=['ptid'],keep='first')

In [4]:
dfsample.assign(fold='semisup_AMD_preinj_sample').to_parquet('/data/amd-data/cera-rpd/detectron2-rpd/datasets/dfUWAMD_dfsemisup_preinj_sample.parquet')

In [ ]:
#find high frequency volumes for longitudnal study
dfcnt = df.groupby(['ptid','eye'])['instance'].count()
ptid= dfcnt[dfcnt>20].reset_index().ptid
df = df[df.ptid.isin(ptid)]
df.astype(str).nunique()
#df.assign(fold='semisup_AMD_long_preinj').to_parquet('/data/amd-data/cera-rpd/detectron2-rpd/datasets/dfUWAMD_dfsemisup_long_preinj.parquet')

# Read in dataframe of interest

In [ ]:
df = pd.read_parquet('dfUWAMD_eval_train_v2_classify.parquet')
df = df.drop(columns = ['slicei','img_path'])
df = df.loc[df.astype(str).drop_duplicates().index]
df.columns


In [2]:
df = pd.read_parquet('dfUWAMD_dfsemisup_long.parquet')
df.columns

Index(['ptid', 'eye', 'age', 'instance', 'diagnosis', 'logmar', 'fold'], dtype='object')

In [6]:
df = pd.read_parquet('/data/amd-data/cera-rpd/detectron2-rpd/datasets/dfUWAMD_dfsemisup_preinj_sample.parquet')
df

,ptid,eye,age,instance,diagnosis,fold
index,,,,,,
45075,4e5103540fcdb59e9d723a3e587b6691a2cd27bb23be7f...,R,85.7550,3029e183d1bd08975904bcdff44422fc23ea2bb3e302a7...,"[AMD, Glaucoma]",semisup_AMD_preinj_sample
19849,f223309310c7f891f1d71e7da5dcc18d44cc38f6b7e15f...,L,51.0308,42aae076ebf09f379bb97d80c0cfc47c88bb5a22460d07...,[AMD],semisup_AMD_preinj_sample
24372,d9befdef5161407446a8906144c380da3c09f89ff3f3bc...,L,71.9425,11cd87c20226279b0017822b508aafc91c7cde74b69ebd...,"[AMD, Inherited Retinal Disease]",semisup_AMD_preinj_sample
8577,84e5d5cd17cfa2f1debfd87ade427ad13128674e9032b1...,R,58.2724,cfab62b2bef9f410add778e9e4582c7b63b5e10d7eaacd...,[AMD],semisup_AMD_preinj_sample
9278,2648d9bed74ec1c27bcc631e66d3e2b3c66440495d9ced...,R,67.5838,fc74a04ad9d6f1d58263d741028d7b1ecea04d8cf9aa4b...,"[Glaucoma, AMD]",semisup_AMD_preinj_sample
...,...,...,...,...,...,...
17886,7f5592146f2c1f9aae11f0e4507744142598bcd85e89a5...,R,70.8419,7b4a45967ad7c10060720745eb9ae5b861f3ad6053d7f7...,[AMD],semisup_AMD_preinj_sample
71668,575b1304ad56af3853ef1cfded3857088c3a9c7aa822b6...,L,70.4367,eaa64038b36ae89664943a5643e76508fdae0c05228934...,[AMD],semisup_AMD_preinj_sample
84068,49bd3b5dba2a400089bc5c9e8b9718de7da8ad377f13b7...,R,79.8987,ececd5f96692b9bc000f20ea432ae224077b2310d8f656...,[AMD],semisup_AMD_preinj_sample


In [9]:

#dfmap = pd.DataFame(columns=['instance','referenceID'])
rootdir = '/data/oct-heyex-data/macoct/'
for ptid in tqdm(df.ptid.unique()): #for each ptid
    path = os.path.join(rootdir,ptid,'macOCT','*','oct.json')
    paths = glob.glob(path)
    for p in paths: #for each instance/eye volume
        d = json.load(open(p))
        dataset.append(d)


100%|██████████| 1428/1428 [00:21<00:00, 66.20it/s] 


In [10]:
dfmap = pd.DataFrame(dataset)
dfmap.columns

Index(['ptid', 'study', 'series', 'age', 'laterality', 'instance', 'thickness',
       'res1', 'res2', 'type', 'octh', 'referenceID', 'volshape'],
      dtype='object')

In [45]:
dfa = df.merge(dfmap[['instance','referenceID','volshape','octh','res2','thickness']],on=['instance'])
dfa = dfa.assign(slicei =dfa.apply(lambda x: list(range(x.volshape[0])),axis=1)).explode(['octh','slicei'])
dfa = dfa.assign(img_path=dfa.apply(lambda x: os.path.join(rootdir,x.ptid,'macOCT',x.referenceID,'oct-{:03d}.png'.format(x.slicei)),axis=1))
#dfa = dfa.assign(fold = 'dfeval_train_v2')
dfa = dfa.assign(msk_path = '')
dfa = dfa.reset_index(drop=True)
dfa.columns


Index(['ptid', 'eye', 'age', 'instance', 'diagnosis', 'fold', 'referenceID',
       'volshape', 'octh', 'res2', 'thickness', 'slicei', 'img_path',
       'msk_path'],
      dtype='object')

In [47]:
pd.set_option('display.max_colwidth',1000)
dfa.loc[:,'ir_path'] = dfa['img_path'].str.replace('oct-....png','ir.png',regex=True)


0        /data/oct-heyex-data/macoct/4e5103540fcdb59e9d723a3e587b6691a2cd27bb23be7f3c225c09ee8d71d7a6/macOCT/2288a15d8e963bae9f7fda3aac76669f2f9e0b96e1e1f4d9b616be0c7171992a/oct-000.png
1        /data/oct-heyex-data/macoct/4e5103540fcdb59e9d723a3e587b6691a2cd27bb23be7f3c225c09ee8d71d7a6/macOCT/2288a15d8e963bae9f7fda3aac76669f2f9e0b96e1e1f4d9b616be0c7171992a/oct-001.png
2        /data/oct-heyex-data/macoct/4e5103540fcdb59e9d723a3e587b6691a2cd27bb23be7f3c225c09ee8d71d7a6/macOCT/2288a15d8e963bae9f7fda3aac76669f2f9e0b96e1e1f4d9b616be0c7171992a/oct-002.png
3        /data/oct-heyex-data/macoct/4e5103540fcdb59e9d723a3e587b6691a2cd27bb23be7f3c225c09ee8d71d7a6/macOCT/2288a15d8e963bae9f7fda3aac76669f2f9e0b96e1e1f4d9b616be0c7171992a/oct-003.png
4        /data/oct-heyex-data/macoct/4e5103540fcdb59e9d723a3e587b6691a2cd27bb23be7f3c225c09ee8d71d7a6/macOCT/2288a15d8e963bae9f7fda3aac76669f2f9e0b96e1e1f4d9b616be0c7171992a/oct-004.png
                                                                      

In [49]:
dfa.to_parquet('/data/amd-data/cera-rpd/detectron2-rpd/datasets/dfUWAMD_dfsemisup_preinj_sample_toserve.parquet')


In [53]:
dfa[['instance','img_path']].set_index('instance').sort_values('img_path').to_dict('series')

{'img_path': instance
 d68f218cbbd47fa716858bd277288812bef9e49eab9913c2495443331c34b8ea    /data/oct-heyex-data/macoct/000e7e0a375f4f04b3d59247f45230b9ed90a2baf36922452a9a85ee6ef4e44f/macOCT/db820d32104cc507d146b2c04f9337f75f54de780e4d74fcd5903c208fb11f45/oct-000.png
 d68f218cbbd47fa716858bd277288812bef9e49eab9913c2495443331c34b8ea    /data/oct-heyex-data/macoct/000e7e0a375f4f04b3d59247f45230b9ed90a2baf36922452a9a85ee6ef4e44f/macOCT/db820d32104cc507d146b2c04f9337f75f54de780e4d74fcd5903c208fb11f45/oct-001.png
 d68f218cbbd47fa716858bd277288812bef9e49eab9913c2495443331c34b8ea    /data/oct-heyex-data/macoct/000e7e0a375f4f04b3d59247f45230b9ed90a2baf36922452a9a85ee6ef4e44f/macOCT/db820d32104cc507d146b2c04f9337f75f54de780e4d74fcd5903c208fb11f45/oct-002.png
 d68f218cbbd47fa716858bd277288812bef9e49eab9913c2495443331c34b8ea    /data/oct-heyex-data/macoct/000e7e0a375f4f04b3d59247f45230b9ed90a2baf36922452a9a85ee6ef4e44f/macOCT/db820d32104cc507d146b2c04f9337f75f54de780e4d74fcd5903c208fb11f45/oct-00

In [48]:
dfa[['ptid', 'eye', 'instance', 'slicei','volshape']].astype('str').nunique()

ptid        1428
eye            2
instance    1428
slicei        61
volshape       1
dtype: int64

In [41]:
from sklearn.model_selection import GroupKFold
splitter = GroupKFold(n_splits=10)
splits = splitter.split(dfa,None,groups=dfa.ptid)

In [42]:
#Check intersection
setlist = []
for i, (_, idx) in enumerate(splits):
    setlist.append(set(dfa.loc[idx]['ptid']))
    dfa.loc[idx,'fold'] = 'fold{}'.format(i+1)
print(set.intersection(*setlist))


set()


In [52]:
dfa[dfa.fold=='fold1'][['ptid','eye','age']].drop_duplicates().groupby(['ptid','eye']).count()

age
ptid                                               eye     
055bdc3ae92212dd5daa3a66358d58283ce6e60dc8c8d9f... L     56
                                                   R     53
251a1bd5290c932d6cc30975aa8806f8318285286c450c6... L     31
                                                   R     32
312b99fe94469e7deb6e5d495723b0b2710560ed4ac0fae... L     20
                                                   R     21
451a0840d96f0270eff6f77ea28de9d08f7c38ce5102a73... L     26
                                                   R     28
6c9b0ab7f453360f55632b4ce94e21e13a683e35388572e... L     25
                                                   R     24
785e3d83e3884be99059c27eb9281741f8cda04620311b2... L     30
8372fdd11a12708c81d60a17da5a4dc8515f89007a526a3... L     34
                                                   R     33
a11344503550b58ab75a62be10976e44699a71a89c60aac... L     16
                                                   R     15
f79399dcc307dff6e327a2fc6cbbf03183154413643fb8a... R     23

In [44]:
dfa.to_parquet('dfsemisup_long_refined.parquet')

In [46]:
from data import rpd_data
import pickle

data = rpd_data(dfa.rename(columns={'instance':'volID'}),grp='fold1',data_has_ann=False)
pickle.dump(data, open("dfsemisup_long_fold1.pk", "wb"))

29585it [02:35, 190.02it/s]


Found 29585 images
Found 0 instances
Found 0 too few vertices


In [49]:
dat = pickle.load(open('dfsemisup_long_fold1.pk','rb'))
len(dat)

603.7755102040817